In [1]:
print("start")
from chdb import session
print("Connecting to database")
chs = session.Session()
print("Creating database")
chs.query("CREATE DATABASE IF NOT EXISTS movie_embeddings ENGINE = Atomic")
print("Creating table")
chs.query("USE movie_embeddings")
chs.query('DROP TABLE IF EXISTS embeddings')
chs.query('DROP TABLE IF EXISTS embeddings_with_title')

print("Creating table with movie titles")

chs.query("""CREATE TABLE embeddings (
      movieId UInt32 NOT NULL,
      embedding Array(Float32) NOT NULL
  ) ENGINE = MergeTree()
  ORDER BY movieId""")

print("Inserting movie embeddings into the database")

# Insert from INFILE and VALUES got stuck either
# chs.query("INSERT INTO embeddings FROM INFILE 'movie_embeddings.csv' FORMAT CSV")
chs.query("INSERT INTO embeddings VALUES (1, [1,2,3,4,5,6,7,8,9,10])")
print(chs.query('SELECT * FROM embeddings LIMIT 5'))

start
Connecting to database
Creating database
Creating table
Creating table with movie titles
Inserting movie embeddings into the database
1,"[1,2,3,4,5,6,7,8,9,10]"

